In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from lib.testing_utilities import get_num_params, boards, visualize_state, random_board
from lib.utilities import config, product
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.ioff()

In [3]:
# Generate 300 random boards for benchmarking
boards = []
np.random.seed(42)
for _ in range(300):
  density = np.random.uniform(0, 0.6) + 0.01
  boards.append(random_board(density))

for board in boards:
  assert board[1].sum() == 1
  
boards = torch.stack(boards)

## Model v1

In [4]:
from lib.model_v1 import TDConway as TDConway_1
model_1 = TDConway_1(config)
get_num_params(model_1)

59,943,809 parameters


In [5]:
%%timeit
with torch.no_grad():
  model_1(boards)

8.09 s ± 547 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
# with torch.no_grad():
#   res = td_conway(boards)
# res

## Model v2

In [10]:
from lib.model_v2 import TDConway as TDConway_2
model_2 = TDConway_2(config)
get_num_params(model_2)

4,529,089 parameters


In [11]:
%%timeit
with torch.no_grad():
  model_2(boards)

1.09 s ± 20.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Test Move Generation

In [ ]:
from lib.testing_utilities import boards, random_board
from lib.move_selection import get_move_options
from lib.timer import Timer
import torch
import numpy as np

In [ ]:
# for i, board in enumerate(boards):
#   timer = Timer()
#   options = get_move_options(board, torch.device('cpu'))
#   print(f'Board {i} - {timer}')

In [ ]:
from lib.memory import get_memory_usage, format_bytes, print_memory_usage

In [ ]:
print_memory_usage()

import gc

print(f'{gc.collect()} items collected')

print_memory_usage()

In [ ]:
bad_boards = []

np.random.seed(42)
timer = Timer()

baseline_memory = get_memory_usage()

for i in range(1, 1001):
  if i % 10 == 0:
    print(f'{i} boards done - {timer} - {format_bytes(memory)}')
    timer = Timer()
    
  board = random_board(0.6)
  
  try:
    get_move_options(board, device = torch.device('cpu'), debug = True)
  except RuntimeError as e:
    bad_boards.append((board, e.args[0]))
  
  memory = get_memory_usage()
  if memory > baseline_memory + 2 * 1000 * 1000 * 1000: #2 GB
    raise RuntimeError('Excessive memory')
    